In [ ]:
import os
from typing import List

from alibabacloud_sls20201230.client import Client as Sls20201230Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_sls20201230 import models as sls_20201230_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient
import os
import pandas as pd
from datetime import datetime, timedelta

import logging

ds_to_run=datetime.now().strftime("%Y%m%d")
ds_yesterday=(datetime.now()-timedelta(days=1)).strftime("%Y%m%d")

# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

ALIBABA_CLOUD_ACCESS_KEY_ID = os.environ["ALIBABA_CLOUD_ACCESS_KEY_ID"]
ALIBABA_CLOUD_ACCESS_KEY_SECRET = os.environ["ALIBABA_CLOUD_ACCESS_KEY_SECRET"]

config = open_api_models.Config(
    access_key_id=ALIBABA_CLOUD_ACCESS_KEY_ID,
    access_key_secret=ALIBABA_CLOUD_ACCESS_KEY_SECRET,
    read_timeout=120 * 1000,
    connect_timeout=10 * 1000,
    no_proxy="cn-hangzhou.log.aliyuncs.com",
)
# Endpoint 请参考 https://api.aliyun.com/product/Sls
config.endpoint = f"cn-hangzhou.log.aliyuncs.com"
sls_client = Sls20201230Client(config)
sls_client._read_timeout = 120 * 1000

In [ ]:
# 获取SLS日志的查询结果
import time

headers = {
    "accept": "application/json",
    "user-agent": "AlibabaCloud API Workbench",
    "x-log-apiversion": "0.6.0",
    "x-log-bodyrawsize": "0",
    "x-log-signaturemethod": "hmac-sha256",
    "content-type": "application/json",
}


def get_sls_data_by_query(
    from_time: datetime,
    to_time: datetime,
    query: str = "",
    project: str = "xianmu-front-end-log",
    logstore: str = "xm-mall",
    retry_time: int = 1,
    line: int = 100,
    offset: int = 0,
) -> pd.DataFrame:
    if retry_time < 0:
        logging.error(f"超过了最多重试次数")
        return None
    if offset % 10000 == 0:
        logging.info(
            f"即将获取数据: =====>from_time:{from_time}, to_time:{to_time}, logstore:{logstore}, query:{query[0:150]}",
        )

    from_ = int(from_time.timestamp())
    to_ = int(to_time.timestamp())

    get_logs_v2headers = sls_20201230_models.GetLogsV2Headers(
        common_headers=headers, accept_encoding="lz4"
    )
    get_logs_v2request = sls_20201230_models.GetLogsV2Request(
        from_=from_,
        to=to_,
        query=query,
        line=line,
        offset=offset,
    )
    runtime = util_models.RuntimeOptions(
        connect_timeout=10 * 1000,
        read_timeout=120 * 1000,
        no_proxy="cn-hangzhou.log.aliyuncs.com",
        max_attempts=2,
    )

    product_view_data = []
    try:
        response = sls_client.get_logs_v2with_options(
            project=project,
            logstore=logstore,
            request=get_logs_v2request,
            headers=get_logs_v2headers,
            runtime=runtime,
        )
        product_view_data = response.body.data
        if offset % 10000 == 0:
            logging.info(f">=====数据条数:{len(product_view_data)}")
        return pd.DataFrame(product_view_data)
    except Exception as error:
        logging.error(
            f"查询SLS失败了,重试:{retry_time},project:{project},logstore:{logstore}, 错误:{error}"
        )
        # 5 秒后重试一次
        time.sleep(5)
        return get_sls_data_by_query(
            from_time=from_time,
            to_time=to_time,
            query=query,
            project=project,
            logstore=logstore,
            retry_time=retry_time - 1,
        )

In [ ]:
# 获取SLS日志的查询结果
# 获取用户的分桶信息


def get_mid_variant_info(from_time: datetime, to_time: datetime):
    user_ab_test_info_df = None
    query = """
    ap:/abStrategy/userExperiments | select phone as cust_phone,uid as cust_id,
        json_extract_scalar(json_extract_scalar(ai, '$.rt'),'$.data["new-home"].variantId') variant_id,
        json_extract_scalar(json_extract_scalar(ai, '$.rt'),'$.data["new-home"].experimentId') experiment_id,
        count(1) request_cnt,
        date_format(min(__time__),'%Y-%m-%d %H:%i:%s') min_time,
        date_format(max(__time__),'%Y-%m-%d %H:%i:%s') max_time
    from log 
    where phone is not null and length(phone)>1 group by 1,2,3,4 
    having variant_id is not null limit 1000000
    """

    user_ab_test_info_df = get_sls_data_by_query(
        from_time=from_time,
        to_time=to_time,
        query=query,
    )
    if user_ab_test_info_df is None:
        logging.error(f"没有获取到商品的曝光数据:{from_time}~{to_time}")
        return
    
    return user_ab_test_info_df

In [ ]:
# 获取用户的点击信息（点击包括了商品详情页的点击、加入购物车的点击、以及立即购买的点击）


def get_user_click_data(
    from_time: datetime = datetime.now() - timedelta(hours=1),
    to_time: datetime = datetime.now(),
    page_name: str = "/home",
) -> pd.DataFrame:
    query = f"""type:cl and pageName:{page_name} and uid not null and sku |
        select uid,bid,cid,pid,sku,pageName,url,date_format(__time__,'%Y-%m-%d %H:%i:%S') as arrive_time 
        from log where uid is not null limit 1000000"""
    # user_click_df = pd.DataFrame()
    # offset = 0

    # while True:
    batch_df = get_sls_data_by_query(
        from_time=from_time, to_time=to_time, query=query
        # , offset=offset
    )

    if batch_df is None or len(batch_df) == 0:
        # if offset == 0:
        #     logging.error(f"没有获取到商品的曝光数据:{from_time}~{to_time}")
        return pd.DataFrame()

    # 删除 'userAgent' 和 'v' 列
    # batch_df = batch_df.drop(
    #     columns=["userAgent", "useragent", "v"], errors="ignore"
    # )

    # 确保 uid 列不为 None、nan 或非数字，并将其转换为 int 类型
    batch_df["uid"] = pd.to_numeric(batch_df["uid"], errors="coerce")
    batch_df["uid"] = batch_df["uid"].fillna(-1)
    batch_df["uid"] = batch_df["uid"].astype(int)
    # 删除任何仍然包含非数字的行
    batch_df = batch_df[batch_df["uid"] != -1]
    return batch_df

        # user_click_df = pd.concat([user_click_df, batch_df], ignore_index=True)

        # if len(batch_df) < 100:
        #     print(
        #         f"from_time:{from_time}, to_time:{to_time}, page_name:{page_name} 已经获取完毕了"
        #     )
        #     break

        # offset = len(user_click_df)
        # if offset % 10000 == 0:
        #     print(f"获取更多数据, 新的offset:{offset}")

    # return user_click_df

In [ ]:
# 这里是获取用户分桶数据并保存

from datetime import datetime, timedelta
from odps_client import write_pandas_df_into_odps

N = 2  # Number of days to loop over
end_date = datetime.now().date()

for i in range(N):
    current_date = end_date - timedelta(days=i)
    from_time = datetime.combine(current_date, datetime.min.time())
    to_time = from_time + timedelta(hours=24)
    ds = from_time.strftime("%Y%m%d")

    print(f"Processing data for {from_time} to {to_time}, ds:{ds}")

    df = get_mid_variant_info(from_time=from_time, to_time=to_time)
    df.drop(columns=["__source__", "__time__"], inplace=True)

    table_name = "summerfarm_ds.temp_mall_new_home_ab_info_di"

    partition_spec = f"ds={ds}"
    is_ok = write_pandas_df_into_odps(
        df=df,
        table_name=table_name,
        partition_spec=partition_spec,
        overwrite=True,
    )

    print(f"save {len(df)} records into table:{table_name}, ds:{ds}")

In [ ]:
# 这里是获取用户点击数据并保存

import os
import pandas as pd
from datetime import datetime, timedelta
from odps_client import write_pandas_df_into_odps


def get_csv_location_by_ds(ds: str) -> str:
    return f"./data/user_click_all_new_df_{ds}.csv"


# 如果CSV文件没有数据，则从get_user_click_data获取数据
def load_user_click_data(ds_to_run: str) -> pd.DataFrame:
    # 尝试从CSV文件读取数据
    csv_file_path = get_csv_location_by_ds(ds_to_run)
    if os.path.exists(csv_file_path):
        df = pd.read_csv(csv_file_path)
        if not df.empty:
            print(f"从CSV文件加载了{len(df)}条记录")
            return df

    print(f"未找到日期:{ds_to_run}的数据")
    return pd.DataFrame()


start_date = datetime(2024, 9, 10).date()
end_date = datetime.now().date()
N = (end_date - start_date).days
print(f"start_date:{start_date}, end_date:{end_date}, N:{N}")

user_click_all_df = pd.DataFrame()

for i in range(N):
    from_time = datetime.combine(start_date + timedelta(i), datetime.min.time())
    to_time = from_time + timedelta(hours=24)
    ds = from_time.strftime("%Y%m%d")
    print(f"正在处理 {from_time} 到 {to_time} 的数据，日期：{ds}")

    ds_click_df = load_user_click_data(ds)
    if ds_click_df.empty:
        for page_name in ["/goods", "/search/goods", "/home", "/goods/category"]:
            print(
                f"正在处理 {from_time} 到 {to_time} 的数据，日期：{ds}, page_name:{page_name}"
            )
            # 如果本地缓存没有，则重新跑一遍(每一个page都需要跑):
            df = get_user_click_data(
                from_time=from_time, to_time=to_time, page_name=page_name
            )
            df["ds"] = ds
            df.drop(columns=["__source__", "__time__"], inplace=True, errors="ignore")
            ds_click_df = pd.concat([ds_click_df, df], ignore_index=True)

        # 如果全部获取完了所有页面的数据，则写入本地缓存
        # ds_click_df.drop_duplicates(
        #     inplace=True,
        #     subset=[
        #         "uid",
        #         "cid",
        #         "sid",
        #         "pageName",
        #         "__tag__:__receive_time__",
        #         "__time_ns_part__",
        #     ],
        # )
        ds_click_df.to_csv(get_csv_location_by_ds(ds=ds), index=False)
        print(
            f"已将 {len(ds_click_df)} 条记录保存到 user_click_all_df，总记录数：{len(user_click_all_df)}，日期：{ds}"
        )
    else:
        print(
            f"从本地缓存中获取到了DS:{ds}的数据,数据条数:{len(ds_click_df)}, pages:{list(ds_click_df['pageName'].unique())}"
        )
    user_click_all_df = pd.concat([user_click_all_df, ds_click_df], ignore_index=True)

In [ ]:
from odps_client import get_odps_sql_result_as_df

user_view_sql="""
SELECT  ds
        ,cust_id
        ,b.variant_id
        ,page_name
        ,COUNT(1) as sku_view_cnt
FROM    summerfarm_tech.dwd_log_mall_di a
LEFT JOIN(
        SELECT cust_id as mid,min(variant_id) variant_id
        FROM summerfarm_ds.temp_mall_new_home_ab_info_di
        WHERE ds>='20240910'
        GROUP BY cust_id
) b ON b.mid=a.cust_id
WHERE   a.ds >= '20240910'
AND     a.spu_id IS NOT NULL
and     a.cust_id is not null
AND     a.page_name in ('/goods','/search/goods','/home','/goods/category')
group by a.ds
        ,a.cust_id
        ,b.variant_id
        ,a.page_name
;
"""

user_view_data_df=get_odps_sql_result_as_df(sql=user_view_sql)

In [ ]:
# Create new column '是否点开购物车卡片'
user_click_all_df['是否点开购物车卡片'] = '否'
user_click_all_df.loc[(user_click_all_df['bid'].str.contains('pid:唤起购买', na=False) | 
                       (user_click_all_df['pid'] == '唤起购买')), '是否点开购物车卡片'] = '是'

user_click_all_df['是否点开商品详情页'] = '否'
user_click_all_df.loc[user_click_all_df['bid'].str.contains('pid:goods,sku:', na=False), '是否点开商品详情页'] = '是'

In [ ]:
user_click_all_clean_df = user_click_all_df[
    [
        "uid",
        "ds",
        "是否点开购物车卡片",
        "是否点开商品详情页",
        "pageName",
    ]
].copy()

In [ ]:
import pandasql
import pandasql.sqldf

user_click_analytics_df = pandasql.sqldf("""
                                       select uid,ds,pageName
                                        ,count(case when `是否点开商品详情页` = '是' then 1 end) as 点开商品详情页次数
                                        ,count(case when `是否点开购物车卡片` = '是' then 1 end) as 点开购物车卡片次数
                                       from user_click_all_clean_df
                                       group by uid,ds,pageName
                                       """)

In [ ]:
# 转换连接键为相同类型（字符串）
user_view_data_df["cust_id"] = user_view_data_df["cust_id"].fillna(-1)
user_view_data_df["cust_id"] = (
    user_view_data_df["cust_id"].astype(str).str.replace(".0", "", regex=False)
)
user_view_data_df["cust_id"] = user_view_data_df["cust_id"].astype(int)
user_view_data_df["ds"] = user_view_data_df["ds"].astype(str)
user_view_data_df["page_name"] = user_view_data_df["page_name"].astype(str)

user_click_analytics_df = user_click_analytics_df[
    user_click_analytics_df["uid"] != "nan"
]

user_click_analytics_df["uid"] = (
    user_click_analytics_df["uid"].astype(str).str.replace(".0", "", regex=False)
)
user_click_analytics_df["uid"] = user_click_analytics_df["uid"].astype(int)
user_click_analytics_df["ds"] = user_click_analytics_df["ds"].astype(str)
user_click_analytics_df["pageName"] = user_click_analytics_df["pageName"].astype(str)

# 合并 user_view_data_df 和 user_click_analytics_df
merged_df = user_view_data_df.merge(
    user_click_analytics_df,
    left_on=["cust_id", "ds", "page_name"],
    right_on=["uid", "ds", "pageName"],
    how="left",
    suffixes=("", "_b"),
)

In [ ]:
merged_df['点开商品详情页次数']=merged_df['点开商品详情页次数'].fillna(0)
merged_df['点开购物车卡片次数']=merged_df['点开购物车卡片次数'].fillna(0)
merged_df['点开商品详情页次数']=merged_df['点开商品详情页次数'].astype(int)
merged_df['点开购物车卡片次数']=merged_df['点开购物车卡片次数'].astype(int)

## 获取用户加入购物车的数据

In [ ]:
def get_user_add_cart_data(from_time: datetime, to_time: datetime) -> pd.DataFrame:
    query = """ap:/shopping/cart/upsert/insert and ai |select 
json_extract_scalar(json_extract_scalar(ai, '$.data'),'$.sku')sku,
json_extract_scalar(json_extract_scalar(ai, '$.data'),'$.quantity')quantity,
json_extract(json_extract_scalar(ai, '$.rt'), '$.status') return_status,
json_extract_scalar(json_extract_scalar(ai, '$.rt'), '$.msg') return_msg,
date_format(__time__, '%Y%m%d') as ds,
uid,cid,sid,pageName,url,bid,ap 
from log limit 10000000"""

    user_add_cart_df = None

    user_add_cart_df = get_sls_data_by_query(
        from_time=from_time,
        to_time=to_time,
        query=query,
    )
    if user_add_cart_df is None:
        logging.error(f"没有获取到用户的加入购物车数据:{from_time}~{to_time}")
        return

    return user_add_cart_df

In [ ]:
# 这里是获取用户加购物车数据并保存

import os
import pandas as pd
from datetime import datetime, timedelta
from odps_client import write_pandas_df_into_odps


def get_add_cart_csv_location_by_ds(ds: str) -> str:
    return f"./data/user_add_cart_all_df_{ds}.csv"


# 如果CSV文件没有数据，则从get_user_click_data获取数据
def load_user_add_cart_data(ds_to_run: str) -> pd.DataFrame:
    # 尝试从CSV文件读取数据
    csv_file_path = get_add_cart_csv_location_by_ds(ds_to_run)
    if os.path.exists(csv_file_path):
        df = pd.read_csv(csv_file_path)
        if not df.empty:
            print(f"从CSV文件加载了{len(df)}条记录")
            return df

    print(f"未找到日期:{ds_to_run}的数据")
    return pd.DataFrame()


start_date = datetime(2024, 9, 10).date()
end_date = datetime.now().date()
N = (end_date - start_date).days
print(f"start_date:{start_date}, end_date:{end_date}, N:{N}")

user_add_cart_all_df = pd.DataFrame()

for i in range(N):
    from_time = datetime.combine(start_date + timedelta(i), datetime.min.time())
    to_time = from_time + timedelta(hours=24)
    ds = from_time.strftime("%Y%m%d")
    print(f"正在处理 {from_time} 到 {to_time} 的加购物车数据，日期：{ds}")

    ds_add_cart_df = load_user_add_cart_data(ds)
    if ds_add_cart_df.empty:
        # 如果本地缓存没有，则重新跑一遍
        for hour_index in range(0, 24, 6):  # 6小时一批次
            print(
                f"正在处理 {from_time} 到 {to_time} 的数据，日期：{ds}, hour_index:{hour_index}"
            )

            from_hour = from_time + timedelta(hours=hour_index)
            to_hour = from_hour + timedelta(hours=6)  # 6小时一个批次
            print(f"=======>from_hour:{from_hour}, to_hour:{to_hour}")

            df = get_user_add_cart_data(from_time=from_hour, to_time=to_hour)
            ds_add_cart_df = pd.concat([ds_add_cart_df, df], ignore_index=True)

        # 如果全部获取完了所有页面的数据，则写入本地缓存
        ds_add_cart_df.drop_duplicates(
            inplace=True,
            subset=["uid", "cid", "sid", "pageName", "__time__", "sku", "quantity"],
        )
        ds_add_cart_df.to_csv(get_add_cart_csv_location_by_ds(ds=ds), index=False)
        print(
            f"已将 {len(ds_add_cart_df)} 条记录保存到 user_add_cart_all_df，总记录数：{len(user_add_cart_all_df)}，日期：{ds}"
        )
    else:
        print(
            f"从本地缓存中获取到了DS:{ds}的数据,数据条数:{len(ds_add_cart_df)}, pages:{list(ds_add_cart_df['pageName'].unique())}"
        )
    user_add_cart_all_df = pd.concat(
        [user_add_cart_all_df, ds_add_cart_df], ignore_index=True
    )

In [ ]:
# 创建加购物车汇总数据
add_cart_summary_df = pandasql.sqldf("""
    SELECT uid, ds, pageName AS page_name,
           COUNT(DISTINCT sku) AS added_sku_cnt,
           SUM(quantity) AS added_quantity,
           COUNT(1) AS add_cart_cnt 
    FROM user_add_cart_all_df 
    GROUP BY uid, ds, pageName
""")

# 转换uid类型为整数
add_cart_summary_df['cust_id'] = add_cart_summary_df['uid'].astype(int)
add_cart_summary_df['added_quantity'] = add_cart_summary_df['added_quantity'].astype(int)
add_cart_summary_df['add_cart_cnt'] = add_cart_summary_df['add_cart_cnt'].astype(int)

merged_df['ds']=merged_df['ds'].astype(str)
add_cart_summary_df['ds']=add_cart_summary_df['ds'].astype(str)

# 左连接merged_df和add_cart_summary_df
merged_all_df = merged_df.merge(
    add_cart_summary_df,
    how='left',
    on=['cust_id', 'ds', 'page_name']
)

# 显示合并后的数据前10行


merged_all_df.head(5)

In [ ]:
click_static_df=pandasql.sqldf(f"""
                  select variant_id
                    ,count(distinct cust_id) 总用户数
                    ,sum(sku_view_cnt) 用户SKU浏览数
                    ,sum(点开商品详情页次数) 点开商品详情页次数
                    ,sum(点开购物车卡片次数) 点开购物车卡片次数
                    ,sum(add_cart_cnt) 加购成功次数API
                    ,sum(added_quantity) SKU总加购件数
                    ,round(sum(added_quantity)/count(distinct cust_id),2) 每用户加购SKU件数
                    ,round(sum(点开购物车卡片次数) * 100.00/sum(sku_view_cnt),3) as 点击CTR_点开加购卡片
                    ,round(sum(点开商品详情页次数) * 100.00/sum(sku_view_cnt),3) as 点击CTR_点到商品详情页
                    ,round(sum(点开购物车卡片次数) * 100.00/sum(sku_view_cnt),3) as 点开加购卡片_转化率
                    ,round(sum(add_cart_cnt) * 100.00/sum(sku_view_cnt),3) as 加购成功API_转化率
                    ,min(ds)||'~'||max(ds) as 日期范围
                  from merged_all_df
                  where ds between '20240910' and '{ds_yesterday}'
                  and page_name in('/search/goods','/goods','/goods/category','/home')
                  group by variant_id
                  """)

click_static_df

In [ ]:
page_df=pandasql.sqldf("""
                  select variant_id,page_name
                    ,count(distinct cust_id) 总用户数
                    ,sum(sku_view_cnt) 用户SKU浏览数
                    ,sum(点开商品详情页次数) 点开商品详情页次数
                    ,sum(点开购物车卡片次数) 点开购物车卡片次数
                    ,sum(add_cart_cnt) 加购成功次数API
                    ,sum(added_quantity) SKU总加购件数
                    ,round(sum(added_quantity)/count(distinct cust_id),2) 每用户加购SKU件数
                    ,round(sum(点开购物车卡片次数) * 100.00/sum(sku_view_cnt),3) as 点击CTR_点开加购卡片
                    ,round(sum(点开商品详情页次数) * 100.00/sum(sku_view_cnt),3) as 点击CTR_点到商品详情页
                    ,round(sum(点开购物车卡片次数) * 100.00/sum(sku_view_cnt),3) as 点开加购卡片_转化率
                    ,round(sum(add_cart_cnt) * 100.00/sum(sku_view_cnt),3) as 加购成功API_转化率
                    ,min(ds)||'~'||max(ds) as 日期范围
                  from merged_all_df
                  where ds>='20240910'
                  and page_name in('/search/goods','/goods','/goods/category','/home')
                  group by variant_id,page_name
                    order by  page_name,variant_id
                  """)

page_df

In [ ]:
ds_to_end=datetime.now().strftime('%m%d')
page_df.to_csv(f"分页面的点击转化数据-0910-{ds_to_end}.csv", index=False)

In [ ]:
temp_mall_new_home_ab_info_di_df=get_odps_sql_result_as_df("""select ds,count(1) total_uv
                                                           ,count(case when variant_id='V1' then 1 end) as v1_users
                                                           ,count(case when variant_id='V2' then 1 end) as v2_users
                                                           ,count(case when variant_id='V3' then 1 end) as v3_users
                                                           ,count(case when variant_id='V4' then 1 end) as v4_users
                                                           from summerfarm_ds.temp_mall_new_home_ab_info_di where ds >= '20240910' group by ds order by ds""")
temp_mall_new_home_ab_info_di_df

In [ ]:
ds_by_yeaterday=(datetime.now()-timedelta(days=1)).strftime("%Y-%m-%d")

order_query=f"""
SELECT  variant_id
        ,COUNT(DISTINCT order_no) order_cnt
        ,COUNT(DISTINCT m_id) 下单用户数
        ,SUM(total_price) 总下单GMV
        ,round(AVG(total_price),2) 订单均价
        ,round(SUM(total_price)/COUNT(DISTINCT m_id),2) 用户平均GMV
        ,MIN(order_time)||'~'||MAX(order_time) as time_range
FROM    summerfarm_tech.ods_orders_df a
LEFT JOIN   (
                SELECT  cust_id
                        ,MIN(variant_id) AS variant_id
                FROM    summerfarm_ds.temp_mall_new_home_ab_info_di
                WHERE   ds >= '20240910'
                GROUP BY cust_id
            ) variant
ON      a.m_id = variant.cust_id
AND     a.ds = MAX_PT("summerfarm_tech.ods_orders_df")
WHERE   a.order_time between '2024-09-10 00:00:00' and '{ds_by_yeaterday} 23:59:59'
AND     a.ds = MAX_PT("summerfarm_tech.ods_orders_df")
AND     a.status in (2,3,6)
GROUP BY variant_id;
"""
orders_df=get_odps_sql_result_as_df(order_query)
orders_df.head(2)

In [ ]:
user_order_query=f"""
SELECT  m_id as cust_id,variant_id
        ,COUNT(DISTINCT order_no) order_cnt
        ,SUM(total_price) 总下单GMV
        ,round(AVG(total_price),2) 订单均价
FROM    summerfarm_tech.ods_orders_df a
LEFT JOIN   (
                SELECT  cust_id
                        ,MIN(variant_id) AS variant_id
                FROM    summerfarm_ds.temp_mall_new_home_ab_info_di
                WHERE   ds >= '20240910'
                GROUP BY cust_id
            ) variant
ON      a.m_id = variant.cust_id
AND     a.ds = MAX_PT("summerfarm_tech.ods_orders_df")
WHERE   a.order_time between '2024-09-10 00:00:00' and '{ds_by_yeaterday} 23:59:59'
AND     a.ds = MAX_PT("summerfarm_tech.ods_orders_df")
AND     a.status in (2,3,6)
GROUP BY variant_id,m_id;
"""

user_orders_df=get_odps_sql_result_as_df(user_order_query)
user_orders_df.head(2)

In [ ]:
# 左连接orders_df到click_static_df
order_merged_result = click_static_df.merge(orders_df, on="variant_id", how="left")

# 重命名一些列名以避免冲突
order_merged_result = order_merged_result.rename(
    columns={
        "总用户数": "总浏览用户数",
        "下单用户数": "总下单用户数",
        "order_cnt": "总下单数",
    }
)

# 选择需要的列并重新排序
final_result = order_merged_result[
    [
        "variant_id",
        "总浏览用户数",
        "用户SKU浏览数",
        "点开商品详情页次数",
        "点开购物车卡片次数",
        "加购成功次数API",
        "SKU总加购件数",
        "每用户加购SKU件数",
        "点击CTR_点开加购卡片",
        "点击CTR_点到商品详情页",
        "点开加购卡片_转化率",
        "加购成功API_转化率",
        "总下单数",
        "总下单用户数",
        "总下单gmv",
        "订单均价",
        "用户平均gmv",
        "日期范围",
    ]
]

# 显示结果
final_result

In [ ]:
final_result.to_csv(f"./商场价格展示优化AB实验结果_0910_{ds_yesterday}.csv", index=False)

## 计算p-value

In [ ]:
print(merged_all_df.columns)
print(merged_all_df['page_name'].unique())

### 先计算分页面的点击转化p-value

In [ ]:
from scipy import stats

user_merged_all_df = pandasql.sqldf(
    """
                                  select cust_id,variant_id,page_name,
                                  sum(sku_view_cnt) as SKU浏览总次数,
                                  sum(点开商品详情页次数) as 商品详情页点击总次数,
                                  sum(点开购物车卡片次数) as 购物车卡片点击总次数,
                                  sum(CAST(COALESCE(added_quantity, 0) AS INTEGER)) as SKU加购总件数,
                                  sum(CAST(COALESCE(add_cart_cnt, 0) AS INTEGER)) as 加购成功总次数
                                  from merged_all_df
                                  group by cust_id,variant_id,page_name
"""
)

In [ ]:
from IPython.core.display import HTML

css = """
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.0.0/dist/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
<style type=\"text/css\">
#abTesting table,#abTesting .table {
    color: #333;
    font-family: unset;
    font-size: 12px;
    line-height: 1.5;
    width: 1024px;
    border-collapse:
    collapse; 
    border-spacing: 0;
    font-family: "SF Pro SC", "SF Pro Text", "SF Pro Icons", "PingFang SC", "Helvetica Neue", "Helvetica", "Arial", sans-serif;
}

tr{
    border-bottom: 1px solid #C1C3D1;
}

tr:nth-child(even) {
    background-color: #F8F8F8;
}

td, th {
    /* border: 1px solid transparent; No more visible border */
    height: 30px;
}

#abTesting table td,#abTesting .table td{
    padding: 0.1rem .75rem;
    vertical-align: middle;
}

th {
    background-color: #DFDFDF; /* Darken header a bit */
    font-weight: bolder;
    min-width: 100px;
    text-align: center;
}
</style>
"""


def display_p_value_below_005(row):
    p_value = row["p_value"]
    color = "black"
    if p_value is not None and p_value <= 0.05:
        color = "red"
    return f"""<span style='font-weight:bolder;color:{color};'>{p_value}</span>"""

In [ ]:
# Convert np.float64 values to regular floats for JSON compatibility
import numpy as np
import json


def convert_np_float64(obj):
    if isinstance(obj, np.float64):
        return round(float(obj), 4)
    return round(obj, 4)


# 计算每组的平均值、中位数和百分位数
def get_stats(group: pd.Series):
    return {
        "avg": convert_np_float64(group.mean()),
        "median": convert_np_float64(group.median()),
        "total_cnt": convert_np_float64(group.count()),
        "total_sum": convert_np_float64(group.sum()),
        "75th_percentile": convert_np_float64(group.quantile(0.75)),
        "90th_percentile": convert_np_float64(group.quantile(0.90)),
        "95th_percentile": convert_np_float64(group.quantile(0.95)),
    }


# 计算两个变体之间指定指标的t检验并返回统计信息的函数
def calculate_p_values(df, metric, variant1="V1", variant2="V2"):
    group1 = df[df["variant_id"] == variant1][metric]
    group2 = df[df["variant_id"] == variant2][metric]

    # 执行双样本t检验
    t_stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)

    # group1_stats = get_stats(group1)
    group2_stats = get_stats(group2)
    group2_stats["p_value"] = convert_np_float64(p_value)
    group2_stats["metric"] = metric
    group2_stats["variant"] = variant2

    return group2_stats


# Function to highlight cells based on p-value
def highlight_p_value(val):
    if pd.notnull(val) and val < 0.05:
        return "background-color: light-purple"
    else:
        return ""


# Example: Calculate p-values for each metric between V1 and other variants
metrics = [
    "SKU浏览总次数",
    "商品详情页点击总次数",
    "购物车卡片点击总次数",
    "SKU加购总件数",
    "加购成功总次数",
]
variants = ["V2", "V3", "V4"]

html_content = css

for page_name in merged_all_df["page_name"].unique():
    print(f"\n\n>>>> 开始计算页面:{page_name} 的p-value:\n")
    df = user_merged_all_df[user_merged_all_df["page_name"] == page_name]
    # Store p-values
    p_values = []

    for metric in metrics:
        group_v1 = df[df["variant_id"] == "V1"][metric]
        v1_stats = get_stats(group_v1)
        v1_stats["p_value"] = None
        v1_stats["metric"] = metric
        v1_stats["variant"] = "V1"
        v1_stats["page_name"] = page_name
        p_values.append(v1_stats)

        for variant in variants:
            p_value = calculate_p_values(df, metric, variant1="V1", variant2=variant)
            p_value["page_name"] = page_name
            p_values.append(p_value)

    df_to_display = pd.DataFrame(p_values)
    display(df_to_display)

    df_to_display["P-value"] = df_to_display.apply(display_p_value_below_005, axis=1)

    html_content = html_content + df_to_display[
        [
            "median",
            "75th_percentile",
            "90th_percentile",
            "95th_percentile",
            "avg",
            "P-value",
            "metric",
            "variant",
            "page_name",
        ]
    ].to_html(escape=False, index=False, classes="table dataframe")


html_content = f"""<html><head><meta charset="UTF-8">
<meta name="title" content="商城价格展示优化AB实验结果_0910-{ds_by_yeaterday}">
</head><body>
<h2>当P-value <= 0.05时表示实验结果统计学显著</h2>
<span>统计学显著时，既可能表示该试验组是好于对照组，也可能是坏于对照组</span>
<div id="abTesting">{html_content}</div></body></html>"""
file_path = f"./商城价格展示优化AB实验结果_{ds_by_yeaterday}.html"

# 保存HTML到本地文件：
with open(file_path, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"写入HTML成功！{file_path}")

## 计算下单的p-value

In [ ]:
print(user_orders_df.columns)

# 确保 '总下单gmv' 和 '订单均价' 列为浮点型
user_orders_df['总下单gmv'] = user_orders_df['总下单gmv'].astype(float)
user_orders_df['订单均价'] = user_orders_df['订单均价'].astype(float)

print("'总下单gmv' 列的数据类型:", user_orders_df['总下单gmv'].dtype)
print("'订单均价' 列的数据类型:", user_orders_df['订单均价'].dtype)

# Example: Calculate p-values for each metric between V1 and other variants
metrics = ["order_cnt", "总下单gmv", "订单均价"]
variants = ["V2", "V3", "V4"]

order_html_content = css

p_values = []

for metric in metrics:
    group_v1 = user_orders_df[user_orders_df["variant_id"] == "V1"][metric]
    v1_stats = get_stats(group_v1)
    v1_stats["p_value"] = None
    v1_stats["metric"] = metric
    v1_stats["variant"] = "V1"
    p_values.append(v1_stats)

    for variant in variants:
        p_value = calculate_p_values(user_orders_df, metric, variant1="V1", variant2=variant)
        p_values.append(p_value)

order_p_value_df = pd.DataFrame(p_values)
display(order_p_value_df)

order_p_value_df["P-value"] = order_p_value_df.apply(display_p_value_below_005, axis=1)

order_html_content = order_html_content + order_p_value_df[
    [
        "median",
        "75th_percentile",
        "90th_percentile",
        "95th_percentile",
        "avg",
        "P-value",
        "metric",
        "variant",
    ]
].to_html(escape=False, index=False, classes="table dataframe")


order_html_content = f"""<html><head><meta charset="UTF-8">
<meta name="title" content="商城价格展示优化AB实验结果_下单转化_0910-{ds_by_yeaterday}">
</head><body>
<h2>当P-value <= 0.05时表示实验结果统计学显著</h2>
<span>统计学显著时，既可能表示该试验组是好于对照组，也可能是坏于对照组</span>
<div id="abTesting">{order_html_content}</div></body></html>"""
file_path = f"./商城价格展示优化AB实验结果_下单转化_{ds_by_yeaterday}.html"

# 保存HTML到本地文件：
with open(file_path, "w", encoding="utf-8") as f:
    f.write(order_html_content)

print(f"写入HTML成功！{file_path}")